In [1]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
import datetime
from datetime import date, timedelta
import sys
import os
import csv

import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,Conv2D,RNN
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
os.chdir('C:\\Users\\720\\Desktop\\fighting\\모의투자')
corplist = pd.read_csv('corplist200_m.csv', encoding='cp949') # , encoding='cp949'
corplist = corplist.drop(corplist.index[123]).reset_index(drop=True)
corplist = corplist.drop(corplist.index[40]).reset_index(drop=True)

In [3]:
kospi200 = corplist.copy()
kospi200

,company,code
0,삼성전자,5930
1,SK하이닉스,660
2,LG화학,51910
3,현대차,5380
4,삼성바이오로직스,207940
...,...,...
130,일양약품,7570
131,JW중외제약,1060
132,SPC삼립,5610
133,삼양사,145990


In [4]:
# 기업별 주가 모음 DB로 연결

def convertcode(code):
    code = str(code)
    code = '0'*(6-len(code))+code
    return code


def getDB(sql,stock_file):
    with sqlite3.connect(stock_file) as conn:
        cur=conn.cursor()
        qry=cur.execute(sql)
        desc=[row[0] for row in qry.description]
        data=qry.fetchall()
    return((desc,data))

def weekday(a):
    days = ['월','화','수','목','금','토','일']
    weekday = days[datetime.date(int(a[0:4]),int(a[5:7]),int(a[8:10])).weekday()]
    return weekday

In [8]:
corplist=[17,18,22,24,25,27,30,39]
code = convertcode(kospi200['code'][27])
code

'010130'

# 회사전체 돌리기 

In [20]:
# 회사전체 돌리기 
for com in range(0,len(corplist)):
    
    code = convertcode(kospi200['code'][com])
    company = kospi200['company'][com]
    
    print("------------",com,"-----",code,"-----",company,"-------------")

    
    stock_file = "stock/"+ code+'.db'

    # 주가 

    sql= "select sdate,sopen,shigh,slow,sclose,svol from stockp "
    mdata=getDB(sql,stock_file)
    df_stock=pd.DataFrame(data=mdata[1],columns=mdata[0])
    df_stock.rename(columns = {'sdate' : 'date'}, inplace = True)
    df_stock = df_stock.sort_values(by=['date'])
    df_stock = df_stock.reset_index(drop=True)
    df_stock['weekday'] = df_stock.apply(lambda x: weekday(x['date']),axis=1)

    # 경제지표

    df_index = pd.read_csv("index.csv", encoding='utf-8')
    df_index['gdp']=df_index['gdp'].astype('float')
    df_index['usd']=df_index['usd'].astype('float')


    # NLP
    os.chdir('C:\\Users\\720\\Desktop\\fighting\\모의투자\\TDM') ##tdmfolder 있는 폴더
    tdmfile_after_n = code+'tdm_chart_after.csv' # 000660tdm_chart_after
    tdmfile_before_n = code+'tdm_chart_before.csv'

    # tdmfile_after = pd.read_csv(tdmfile_after_n, encoding='cp949') ## 이건 필요하면 쓰자구
    tdmfile_before = pd.read_csv(tdmfile_before_n, encoding='cp949')
    nlp_len = len(tdmfile_before)-1

    # 재무데이터
    os.chdir('C:\\Users\\720\\Desktop\\fighting\\모의투자')
    df_finance = pd.read_csv('./재무제표/지표/finance_{}.csv'.format(code))
    df_finance['periods']=['2018-03-31','2018-06-30','2018-09-30','2018-12-30',
                  '2019-03-31','2019-06-30','2019-09-30','2019-12-30',
                  '2020-03-31','2020-06-30','2020-09-29','2020-12-30'] # 2020-09-30 추석, 20xx-12-30 장 마감
    df_finance['weekday'] = df_finance.apply(lambda x: weekday(x['periods']), axis=1)

    for i in range(len(df_finance)):
        if df_finance['weekday'][i] == "토":
            date = datetime.datetime(int(df_finance['periods'][i][0:4]),int(df_finance['periods'][i][5:7]),int(df_finance['periods'][i][8:10]))-timedelta(1)
            date = date.strftime('%Y-%m-%d')
            df_finance['periods'][i] = date
            df_finance['weekday'][i] = "금"
        elif df_finance['weekday'][i] == "일":
            date = datetime.datetime(int(df_finance['periods'][i][0:4]),int(df_finance['periods'][i][5:7]),int(df_finance['periods'][i][8:10]))-timedelta(2)
            date = date.strftime('%Y-%m-%d')
            df_finance['periods'][i] = date
            df_finance['weekday'][i] = "금"
        else: pass

    # stock
    try:
        start = df_stock[df_stock['date']=='2018-01-01'].index[0]
    except:
        start = 0
    end = df_stock[df_stock['date']=='2020-12-30'].index[0]

    df = pd.merge(df_stock[start:end+1], df_finance, left_on='date', right_on='periods', how='outer')

    df = pd.merge(df, df_index, on='date', how='left')

    df = df.fillna(method='backfill')
    df_bnlp = df.copy()

    # NLP_merge pd.merge(df_before, tdmfile_before, left_on='sdate', right_on='sdate', how='left').fillna(0.).sort_values(by='sdate')
    df = pd.merge(df, tdmfile_before, left_on='date', right_on='sdate', how='left').fillna(0.)

    df.drop(['sdate','weekday_x','weekday_y','code','periods'],axis=1,inplace=True)

    df=df.set_index('date')

    # min-max
    from sklearn.preprocessing import MinMaxScaler
    mxs=MinMaxScaler()
    mxs.fit(df)
    outv=mxs.transform(df)
    msdf=pd.DataFrame(outv,index=df.index,columns=df.columns)

    # =========================================== 모델 돌리기 ===============================================

    df_result = pd.DataFrame(columns=['chart_RMSE','bs_RMSE','index_RMSE','nlp_RMSE','bs_index_RMSE', 'bs_nlp_RMSE', 'index_nlp_RMSE', 'bs_index_nlp_RMSE'])

    for s in range(5):
        # 차트만 넣은 모델 돌리기

        winsize=3
        x_data=[]
        y_data=[]

        for i in range(len(msdf)-winsize):
            x_data.append(msdf.iloc[i:winsize+i,0:5])
            y_data.append(msdf.iloc[winsize+i,3])

        x_data=np.array(x_data).astype('float32')
        y_data=np.array(y_data).astype('float32')

        train_num = 550

        x_train = x_data[:train_num]
        x_val = x_data[train_num:]
        y_train = y_data[:train_num]
        y_val = y_data[train_num:]    

        model=Sequential()
        model.add(LSTM(128,return_sequences=False, input_shape=(winsize,5,)))
        model.add(Dense(64,activation ="linear"))
        model.add(Dense(1,activation="sigmoid"))
        model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
        print('predict model:', model.summary())

                 #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
                 #cosine_proximity는 안나올 수도 있다.
        model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
        predict_chart=model.predict(x_val)



        # NLP만 넣은 모델돌리기
        winsize=3
        x_data=[]
        y_data=[]
        msdf_NLP = msdf.drop(['f01', 'f03', 'f04', 'f05',
               'f06', 'f07', 'f08', 'f11', 'f12', 'f14', 'f15', 'f21', 'f22', 'f23',
               'f31', 'f33', 'f34', 'f41', 'f42', 'f51', 'f52', 'usd', 'kbond', 'cd', 'gdp', 'kospi'], axis=1)

        for i in range(len(msdf)-winsize):
            x_data.append(msdf_NLP.iloc[i:winsize+i,:nlp_len+5])
            y_data.append(msdf_NLP.iloc[winsize+i,3])

        x_data=np.array(x_data).astype('float32')
        y_data=np.array(y_data).astype('float32')

        train_num = 550

        x_train = x_data[:train_num]
        x_val = x_data[train_num:]
        y_train = y_data[:train_num]
        y_val = y_data[train_num:]    

        model=Sequential()
        model.add(LSTM(128,return_sequences=False, input_shape=(winsize,nlp_len+5,)))
        model.add(Dense(64,activation ="linear"))
        model.add(Dense(1,activation="sigmoid"))
        model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
        print('predict model:', model.summary())

                 #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
                 #cosine_proximity는 안나올 수도 있다.
        model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))

        predict_nlp=model.predict(x_val)



        # BS만 넣은 모델 돌리기
        winsize=3
        x_data=[]
        y_data=[]

        for i in range(len(msdf)-winsize):
            x_data.append(msdf.iloc[i:winsize+i,:26])
            y_data.append(msdf.iloc[winsize+i,3])
        x_data=np.array(x_data).astype('float32')
        y_data=np.array(y_data).astype('float32')

        train_num = 550

        x_train = x_data[:train_num]
        x_val = x_data[train_num:]
        y_train = y_data[:train_num]
        y_val = y_data[train_num:]

        model=Sequential()
        model.add(LSTM(128,return_sequences=False, input_shape=(winsize,26,)))
        model.add(Dense(64,activation ="linear"))
        model.add(Dense(1,activation="sigmoid"))
        model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
        print('predict model:', model.summary())
        #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
        #cosine_proximity는 안나올 수도 있다.
        model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
        predict_bs=model.predict(x_val)


        # 경제지표만 넣은 모델 돌리기
        winsize=3
        x_data=[]
        y_data=[]
        msdf_index = msdf.drop(['f01', 'f03', 'f04', 'f05',
               'f06', 'f07', 'f08', 'f11', 'f12', 'f14', 'f15', 'f21', 'f22', 'f23',
               'f31', 'f33', 'f34', 'f41', 'f42', 'f51', 'f52'], axis=1)
        for i in range(len(msdf)-winsize):
            x_data.append(msdf_index.iloc[i:winsize+i,:10])
            y_data.append(msdf_index.iloc[winsize+i,3])
        x_data=np.array(x_data).astype('float32')
        y_data=np.array(y_data).astype('float32')

        train_num = 550

        x_train = x_data[:train_num]
        x_val = x_data[train_num:]
        y_train = y_data[:train_num]
        y_val = y_data[train_num:]

        model=Sequential()
        model.add(LSTM(128,return_sequences=False, input_shape=(winsize,10,)))
        model.add(Dense(64,activation ="linear"))
        model.add(Dense(1,activation="sigmoid"))
        model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
        print('predict model:', model.summary())
        #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
        #cosine_proximity는 안나올 수도 있다.
        model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
        predict_index=model.predict(x_val)


        # BS & INDEX 넣은 모델 돌리기
        winsize=3
        x_data=[]
        y_data=[]

        for i in range(len(msdf)-winsize):
            x_data.append(msdf.iloc[i:winsize+i,:31])
            y_data.append(msdf.iloc[winsize+i,3])
        x_data=np.array(x_data).astype('float32')
        y_data=np.array(y_data).astype('float32')

        train_num = 550

        x_train = x_data[:train_num]
        x_val = x_data[train_num:]
        y_train = y_data[:train_num]
        y_val = y_data[train_num:]

        model=Sequential()
        model.add(LSTM(128,return_sequences=False, input_shape=(winsize,31,)))
        model.add(Dense(64,activation ="linear"))
        model.add(Dense(1,activation="sigmoid"))
        model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
        print('predict model:', model.summary())
        #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
        #cosine_proximity는 안나올 수도 있다.
        model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
        predict_bs_index=model.predict(x_val)


        # BS & NLP 넣은 모델 돌리기
        winsize=3
        x_data=[]
        y_data=[]
        msdf_BN = msdf.drop(['usd', 'kbond', 'cd', 'gdp', 'kospi'], axis=1)

        for i in range(len(msdf)-winsize):
            x_data.append(msdf_BN.iloc[i:winsize+i,:nlp_len+26])
            y_data.append(msdf_BN.iloc[winsize+i,3])
        x_data=np.array(x_data).astype('float32')
        y_data=np.array(y_data).astype('float32')

        train_num = 550

        x_train = x_data[:train_num]
        x_val = x_data[train_num:]
        y_train = y_data[:train_num]
        y_val = y_data[train_num:]

        model=Sequential()
        model.add(LSTM(128,return_sequences=False, input_shape=(winsize,nlp_len+26,)))
        model.add(Dense(64,activation ="linear"))
        model.add(Dense(1,activation="sigmoid"))
        model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
        print('predict model:', model.summary())
        #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
        #cosine_proximity는 안나올 수도 있다.
        model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
        predict_bs_nlp=model.predict(x_val)


        # INDEX & NLP 넣은 모델 돌리기
        winsize=3
        x_data=[]
        y_data=[]
        msdf_IN = msdf.drop(['f01', 'f03', 'f04', 'f05',
               'f06', 'f07', 'f08', 'f11', 'f12', 'f14', 'f15', 'f21', 'f22', 'f23',
               'f31', 'f33', 'f34', 'f41', 'f42', 'f51',], axis=1)


        for i in range(len(msdf)-winsize):
            x_data.append(msdf_IN.iloc[i:winsize+i,:nlp_len+10])
            y_data.append(msdf_IN.iloc[winsize+i,3])
        x_data=np.array(x_data).astype('float32')
        y_data=np.array(y_data).astype('float32')

        train_num = 550

        x_train = x_data[:train_num]
        x_val = x_data[train_num:]
        y_train = y_data[:train_num]
        y_val = y_data[train_num:]

        model=Sequential()
        model.add(LSTM(128,return_sequences=False, input_shape=(winsize,nlp_len+10,)))
        model.add(Dense(64,activation ="linear"))
        model.add(Dense(1,activation="sigmoid"))
        model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
        print('predict model:', model.summary())
        #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
        #cosine_proximity는 안나올 수도 있다.
        model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
        predict_index_nlp=model.predict(x_val)

        # BS & INDEX & NLP 넣은 모델 돌리기
        winsize=3
        x_data=[]
        y_data=[]

        for i in range(len(msdf)-winsize):
            x_data.append(msdf.iloc[i:winsize+i,:nlp_len+31])
            y_data.append(msdf.iloc[winsize+i,3])
        x_data=np.array(x_data).astype('float32')
        y_data=np.array(y_data).astype('float32')

        train_num = 550

        x_train = x_data[:train_num]
        x_val = x_data[train_num:]
        y_train = y_data[:train_num]
        y_val = y_data[train_num:]

        model=Sequential()
        model.add(LSTM(128,return_sequences=False, input_shape=(winsize,nlp_len+31,)))
        model.add(Dense(64,activation ="linear"))
        model.add(Dense(1,activation="sigmoid"))
        model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
        print('predict model:', model.summary())
        #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
        #cosine_proximity는 안나올 수도 있다.
        model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
        predict_bs_index_nlp=model.predict(x_val)




        # 종가
        close_minmax=msdf['sclose'].values[550:]

        # RMSE 산출
        chart_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_chart)).mean())
        bs_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_bs)).mean())
        index_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_index)).mean())
        nlp_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_nlp)).mean())
        bs_index_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_bs_index)).mean())
        bs_nlp_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_bs_nlp)).mean())
        index_nlp_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_index_nlp)).mean())
        bs_index_nlp_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_bs_index_nlp)).mean())

        # bs_nlp_RMSE
        # bs_index_nlp_RMSE
        # index_nlp_RMSE
        # nlp_RMSE

        # 결과값 데이터프레임화
        df_result.loc[s,:] = [chart_RMSE,bs_RMSE,index_RMSE,nlp_RMSE, bs_index_RMSE, bs_nlp_RMSE, index_nlp_RMSE, bs_index_nlp_RMSE]

    df_result2 = pd.DataFrame(data=[df_result.mean().values], columns=['chart_RMSE','bs_RMSE','index_RMSE','nlp_RMSE','bs_index_RMSE', 'bs_nlp_RMSE', 'index_nlp_RMSE', 'bs_index_nlp_RMSE'])
    df_result2.to_csv('df_result_{}.csv'.format(code), index=False)

------------ 0 ----- 005930 ----- 삼성전자 -------------


<ipython-input-20-bf522f6dbb2f>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_finance['periods'][i] = date
<ipython-input-20-bf522f6dbb2f>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_finance['weekday'][i] = "금"
<ipython-input-20-bf522f6dbb2f>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_finance['periods'][i] = date
<ipython-input-20-bf522f6dbb2f>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               68608     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 76,929
Trainable params: 76,929
Non-trainable params: 0
_________________________________________________________________
predict model: None
Epoch 1/20
18/18 [==============================] - 3s 50ms/step - loss: 0.6175 - cosine_proximity: 0.9997 - val_loss: 0.3557 - val_cosine_proximity: 1.0000
Epoch 2/20
18/18 [==============================] - 0s 6ms/step - loss: 0.3115 - cosine_proximity: 0.9982 - val_loss: 0.0494 - val_cosine_proximity: 1.0000
Epoch 3/20
18/18 [==============================]

18/18 [==============================] - 4s 49ms/step - loss: 0.4842 - cosine_proximity: 0.9979 - val_loss: 0.0670 - val_cosine_proximity: 1.0000
Epoch 2/20
18/18 [==============================] - 0s 7ms/step - loss: 0.1676 - cosine_proximity: 0.9996 - val_loss: 0.0471 - val_cosine_proximity: 1.0000
Epoch 3/20
18/18 [==============================] - 0s 6ms/step - loss: 0.1297 - cosine_proximity: 0.9984 - val_loss: 0.0440 - val_cosine_proximity: 1.0000
Epoch 4/20
18/18 [==============================] - 0s 6ms/step - loss: 0.1217 - cosine_proximity: 0.9990 - val_loss: 0.0450 - val_cosine_proximity: 1.0000
Epoch 5/20
18/18 [==============================] - 0s 6ms/step - loss: 0.1187 - cosine_proximity: 0.9993 - val_loss: 0.0456 - val_cosine_proximity: 1.0000
Epoch 6/20
18/18 [==============================] - 0s 5ms/step - loss: 0.1085 - cosine_proximity: 0.9990 - val_loss: 0.0443 - val_cosine_proximity: 1.0000
Epoch 7/20
18/18 [==============================] - 0s 6ms/step - loss: 0.

18/18 [==============================] - 0s 11ms/step - loss: 0.1204 - cosine_proximity: 0.9988 - val_loss: 0.0471 - val_cosine_proximity: 1.0000
Epoch 6/20
18/18 [==============================] - 0s 13ms/step - loss: 0.1125 - cosine_proximity: 0.9942 - val_loss: 0.0462 - val_cosine_proximity: 1.0000
Epoch 7/20
18/18 [==============================] - 0s 11ms/step - loss: 0.1223 - cosine_proximity: 0.9976 - val_loss: 0.0458 - val_cosine_proximity: 1.0000
Epoch 8/20
18/18 [==============================] - 0s 11ms/step - loss: 0.1130 - cosine_proximity: 0.9986 - val_loss: 0.0454 - val_cosine_proximity: 1.0000
Epoch 9/20
18/18 [==============================] - 0s 11ms/step - loss: 0.1178 - cosine_proximity: 0.9991 - val_loss: 0.0452 - val_cosine_proximity: 1.0000
Epoch 10/20
18/18 [==============================] - 0s 11ms/step - loss: 0.1167 - cosine_proximity: 0.9994 - val_loss: 0.0448 - val_cosine_proximity: 1.0000
Epoch 11/20
18/18 [==============================] - 0s 11ms/step - 

...

18/18 [==============================] - 3s 37ms/step - loss: 0.6392 - cosine_proximity: 1.0000 - val_loss: 0.6571 - val_cosine_proximity: 0.9963
Epoch 2/20
18/18 [==============================] - 0s 8ms/step - loss: 0.6019 - cosine_proximity: 1.0000 - val_loss: 0.6621 - val_cosine_proximity: 0.9963
Epoch 3/20
18/18 [==============================] - 0s 8ms/step - loss: 0.6064 - cosine_proximity: 1.0000 - val_loss: 0.6588 - val_cosine_proximity: 0.9963
Epoch 4/20
18/18 [==============================] - 0s 8ms/step - loss: 0.6063 - cosine_proximity: 1.0000 - val_loss: 0.6557 - val_cosine_proximity: 0.9963
Epoch 5/20
18/18 [==============================] - 0s 8ms/step - loss: 0.6016 - cosine_proximity: 1.0000 - val_loss: 0.6554 - val_cosine_proximity: 0.9963
Epoch 6/20
18/18 [==============================] - 0s 8ms/step - loss: 0.6015 - cosine_proximity: 1.0000 - val_loss: 0.6528 - val_cosine_proximity: 0.9963
Epoch 7/20
18/18 [==============================] - 0s 8ms/step - loss: 0.

18/18 [==============================] - 0s 6ms/step - loss: 0.6040 - cosine_proximity: 1.0000 - val_loss: 0.6408 - val_cosine_proximity: 0.9963
Epoch 6/20
18/18 [==============================] - 0s 6ms/step - loss: 0.6048 - cosine_proximity: 1.0000 - val_loss: 0.6292 - val_cosine_proximity: 0.9963
Epoch 7/20
18/18 [==============================] - 0s 6ms/step - loss: 0.5989 - cosine_proximity: 1.0000 - val_loss: 0.6217 - val_cosine_proximity: 0.9963
Epoch 8/20
18/18 [==============================] - 0s 6ms/step - loss: 0.6046 - cosine_proximity: 1.0000 - val_loss: 0.6208 - val_cosine_proximity: 0.9963
Epoch 9/20
18/18 [==============================] - 0s 6ms/step - loss: 0.6050 - cosine_proximity: 1.0000 - val_loss: 0.6083 - val_cosine_proximity: 0.9963
Epoch 10/20
18/18 [==============================] - 0s 6ms/step - loss: 0.6050 - cosine_proximity: 1.0000 - val_loss: 0.6099 - val_cosine_proximity: 0.9963
Epoch 11/20
18/18 [==============================] - 0s 5ms/step - loss: 0

18/18 [==============================] - 0s 8ms/step - loss: 0.6037 - cosine_proximity: 1.0000 - val_loss: 0.6496 - val_cosine_proximity: 0.9963
Epoch 10/20
18/18 [==============================] - 0s 8ms/step - loss: 0.5999 - cosine_proximity: 1.0000 - val_loss: 0.6489 - val_cosine_proximity: 0.9963
Epoch 11/20
18/18 [==============================] - 0s 8ms/step - loss: 0.6047 - cosine_proximity: 1.0000 - val_loss: 0.6471 - val_cosine_proximity: 0.9963
Epoch 12/20
18/18 [==============================] - 0s 9ms/step - loss: 0.6016 - cosine_proximity: 1.0000 - val_loss: 0.6460 - val_cosine_proximity: 0.9963
Epoch 13/20
18/18 [==============================] - 0s 8ms/step - loss: 0.6042 - cosine_proximity: 1.0000 - val_loss: 0.6451 - val_cosine_proximity: 0.9963
Epoch 14/20
18/18 [==============================] - 0s 8ms/step - loss: 0.6019 - cosine_proximity: 1.0000 - val_loss: 0.6459 - val_cosine_proximity: 0.9963
Epoch 15/20
18/18 [==============================] - 0s 8ms/step - los

18/18 [==============================] - 0s 4ms/step - loss: 0.6018 - cosine_proximity: 1.0000 - val_loss: 0.6049 - val_cosine_proximity: 0.9963
Epoch 14/20
18/18 [==============================] - 0s 4ms/step - loss: 0.5973 - cosine_proximity: 1.0000 - val_loss: 0.6025 - val_cosine_proximity: 0.9963
Epoch 15/20
18/18 [==============================] - 0s 5ms/step - loss: 0.6043 - cosine_proximity: 1.0000 - val_loss: 0.5985 - val_cosine_proximity: 0.9963
Epoch 16/20
18/18 [==============================] - 0s 5ms/step - loss: 0.6000 - cosine_proximity: 1.0000 - val_loss: 0.5939 - val_cosine_proximity: 0.9963
Epoch 17/20
18/18 [==============================] - 0s 6ms/step - loss: 0.6023 - cosine_proximity: 1.0000 - val_loss: 0.5933 - val_cosine_proximity: 0.9963
Epoch 18/20
18/18 [==============================] - 0s 5ms/step - loss: 0.6036 - cosine_proximity: 1.0000 - val_loss: 0.5917 - val_cosine_proximity: 0.9963
Epoch 19/20
18/18 [==============================] - 0s 5ms/step - los

18/18 [==============================] - 0s 8ms/step - loss: 0.6077 - cosine_proximity: 1.0000 - val_loss: 0.6353 - val_cosine_proximity: 0.9963
Epoch 18/20
18/18 [==============================] - 0s 8ms/step - loss: 0.6046 - cosine_proximity: 1.0000 - val_loss: 0.6353 - val_cosine_proximity: 0.9963
Epoch 19/20
18/18 [==============================] - 0s 8ms/step - loss: 0.5976 - cosine_proximity: 1.0000 - val_loss: 0.6346 - val_cosine_proximity: 0.9963
Epoch 20/20
18/18 [==============================] - 0s 8ms/step - loss: 0.6066 - cosine_proximity: 1.0000 - val_loss: 0.6336 - val_cosine_proximity: 0.9963


# 개별 회사 1개씩 돌리기

In [93]:
os.chdir('C:\\Users\\pcsea\\work1\\Self_study\\Company_news_PJ\\fdata\\fighting')
#회사 하나씩
company = "빙그레"

code= corplist[corplist['company']==company]['code'].values[0]
code=convertcode(code)

stock_file = "stock/"+ code+'.db'



# 주가 

sql= "select sdate,sopen,shigh,slow,sclose,svol from stockp "
mdata=getDB(sql,stock_file)
df_stock=pd.DataFrame(data=mdata[1],columns=mdata[0])
df_stock.rename(columns = {'sdate' : 'date'}, inplace = True)
df_stock = df_stock.sort_values(by=['date'])
df_stock = df_stock.reset_index(drop=True)
df_stock['weekday'] = df_stock.apply(lambda x: weekday(x['date']),axis=1)

# 경제지표

df_index = pd.read_csv("index.csv", encoding='utf-8')
df_index['gdp']=df_index['gdp'].astype('float')
df_index['usd']=df_index['usd'].astype('float')


# NLP
os.chdir('C:\\Users\\pcsea\\work1\\Self_study\\Company_news_PJ\\fdata\\fighting\\TDM') ##tdmfolder 있는 폴더
tdmfile_after_n = code+'tdm_chart_after.csv' # 000660tdm_chart_after
tdmfile_before_n = code+'tdm_chart_before.csv'

# tdmfile_after = pd.read_csv(tdmfile_after_n, encoding='cp949') ## 이건 필요하면 쓰자구
tdmfile_before = pd.read_csv(tdmfile_before_n, encoding='cp949')
nlp_len = len(tdmfile_before)-1

# 재무데이터
os.chdir('C:\\Users\\pcsea\\work1\\Self_study\\Company_news_PJ\\fdata\\fighting')
df_finance = pd.read_csv('./재무제표/지표/finance_{}.csv'.format(code))
df_finance['periods']=['2018-03-31','2018-06-30','2018-09-30','2018-12-30',
              '2019-03-31','2019-06-30','2019-09-30','2019-12-30',
              '2020-03-31','2020-06-30','2020-09-29','2020-12-30'] # 2020-09-30 추석, 20xx-12-30 장 마감
df_finance['weekday'] = df_finance.apply(lambda x: weekday(x['periods']), axis=1)

for i in range(len(df_finance)):
    if df_finance['weekday'][i] == "토":
        date = datetime.datetime(int(df_finance['periods'][i][0:4]),int(df_finance['periods'][i][5:7]),int(df_finance['periods'][i][8:10]))-timedelta(1)
        date = date.strftime('%Y-%m-%d')
        df_finance['periods'][i] = date
        df_finance['weekday'][i] = "금"
    elif df_finance['weekday'][i] == "일":
        date = datetime.datetime(int(df_finance['periods'][i][0:4]),int(df_finance['periods'][i][5:7]),int(df_finance['periods'][i][8:10]))-timedelta(2)
        date = date.strftime('%Y-%m-%d')
        df_finance['periods'][i] = date
        df_finance['weekday'][i] = "금"
    else: pass
    
# stock
try:
    start = df_stock[df_stock['date']=='2018-01-01'].index[0]
except:
    start = 0
end = df_stock[df_stock['date']=='2020-12-30'].index[0]

df = pd.merge(df_stock[start:end+1], df_finance, left_on='date', right_on='periods', how='outer')

df = pd.merge(df, df_index, on='date', how='left')

df = df.fillna(method='backfill')
df_bnlp = df.copy()

# NLP_merge pd.merge(df_before, tdmfile_before, left_on='sdate', right_on='sdate', how='left').fillna(0.).sort_values(by='sdate')
df = pd.merge(df, tdmfile_before, left_on='date', right_on='sdate', how='left').fillna(0.)

df.drop(['sdate','weekday_x','weekday_y','code','periods'],axis=1,inplace=True)

df=df.set_index('date')

# min-max
from sklearn.preprocessing import MinMaxScaler
mxs=MinMaxScaler()
mxs.fit(df)
outv=mxs.transform(df)
msdf=pd.DataFrame(outv,index=df.index,columns=df.columns)

# =========================================== 모델 돌리기 ===============================================

df_result = pd.DataFrame(columns=['chart_RMSE','bs_RMSE','index_RMSE','nlp_RMSE','bs_index_RMSE', 'bs_nlp_RMSE', 'index_nlp_RMSE', 'bs_index_nlp_RMSE'])

for s in range(5):
    # 차트만 넣은 모델 돌리기

    winsize=3
    x_data=[]
    y_data=[]

    for i in range(len(msdf)-winsize):
        x_data.append(msdf.iloc[i:winsize+i,0:5])
        y_data.append(msdf.iloc[winsize+i,3])

    x_data=np.array(x_data).astype('float32')
    y_data=np.array(y_data).astype('float32')

    train_num = 550

    x_train = x_data[:train_num]
    x_val = x_data[train_num:]
    y_train = y_data[:train_num]
    y_val = y_data[train_num:]    

    model=Sequential()
    model.add(LSTM(128,return_sequences=False, input_shape=(winsize,5,)))
    model.add(Dense(64,activation ="linear"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
    print('predict model:', model.summary())

             #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
             #cosine_proximity는 안나올 수도 있다.
    model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
    predict_chart=model.predict(x_val)

    
    
    # NLP만 넣은 모델돌리기
    winsize=3
    x_data=[]
    y_data=[]
    msdf_NLP = msdf.drop(['f01', 'f03', 'f04', 'f05',
           'f06', 'f07', 'f08', 'f11', 'f12', 'f14', 'f15', 'f21', 'f22', 'f23',
           'f31', 'f33', 'f34', 'f41', 'f42', 'f51', 'f52', 'usd', 'kbond', 'cd', 'gdp', 'kospi'], axis=1)

    for i in range(len(msdf)-winsize):
        x_data.append(msdf_NLP.iloc[i:winsize+i,:nlp_len+5])
        y_data.append(msdf_NLP.iloc[winsize+i,3])

    x_data=np.array(x_data).astype('float32')
    y_data=np.array(y_data).astype('float32')

    train_num = 550

    x_train = x_data[:train_num]
    x_val = x_data[train_num:]
    y_train = y_data[:train_num]
    y_val = y_data[train_num:]    

    model=Sequential()
    model.add(LSTM(128,return_sequences=False, input_shape=(winsize,nlp_len+5,)))
    model.add(Dense(64,activation ="linear"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
    print('predict model:', model.summary())

             #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
             #cosine_proximity는 안나올 수도 있다.
    model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))

    predict_nlp=model.predict(x_val)
    
    

    # BS만 넣은 모델 돌리기
    winsize=3
    x_data=[]
    y_data=[]

    for i in range(len(msdf)-winsize):
        x_data.append(msdf.iloc[i:winsize+i,:26])
        y_data.append(msdf.iloc[winsize+i,3])
    x_data=np.array(x_data).astype('float32')
    y_data=np.array(y_data).astype('float32')

    train_num = 550

    x_train = x_data[:train_num]
    x_val = x_data[train_num:]
    y_train = y_data[:train_num]
    y_val = y_data[train_num:]

    model=Sequential()
    model.add(LSTM(128,return_sequences=False, input_shape=(winsize,26,)))
    model.add(Dense(64,activation ="linear"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
    print('predict model:', model.summary())
    #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
    #cosine_proximity는 안나올 수도 있다.
    model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
    predict_bs=model.predict(x_val)

    
    # 경제지표만 넣은 모델 돌리기
    winsize=3
    x_data=[]
    y_data=[]
    msdf_index = msdf.drop(['f01', 'f03', 'f04', 'f05',
           'f06', 'f07', 'f08', 'f11', 'f12', 'f14', 'f15', 'f21', 'f22', 'f23',
           'f31', 'f33', 'f34', 'f41', 'f42', 'f51', 'f52'], axis=1)
    for i in range(len(msdf)-winsize):
        x_data.append(msdf_index.iloc[i:winsize+i,:10])
        y_data.append(msdf_index.iloc[winsize+i,3])
    x_data=np.array(x_data).astype('float32')
    y_data=np.array(y_data).astype('float32')

    train_num = 550

    x_train = x_data[:train_num]
    x_val = x_data[train_num:]
    y_train = y_data[:train_num]
    y_val = y_data[train_num:]

    model=Sequential()
    model.add(LSTM(128,return_sequences=False, input_shape=(winsize,10,)))
    model.add(Dense(64,activation ="linear"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
    print('predict model:', model.summary())
    #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
    #cosine_proximity는 안나올 수도 있다.
    model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
    predict_index=model.predict(x_val)

    
    # BS & INDEX 넣은 모델 돌리기
    winsize=3
    x_data=[]
    y_data=[]

    for i in range(len(msdf)-winsize):
        x_data.append(msdf.iloc[i:winsize+i,:31])
        y_data.append(msdf.iloc[winsize+i,3])
    x_data=np.array(x_data).astype('float32')
    y_data=np.array(y_data).astype('float32')

    train_num = 550

    x_train = x_data[:train_num]
    x_val = x_data[train_num:]
    y_train = y_data[:train_num]
    y_val = y_data[train_num:]

    model=Sequential()
    model.add(LSTM(128,return_sequences=False, input_shape=(winsize,31,)))
    model.add(Dense(64,activation ="linear"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
    print('predict model:', model.summary())
    #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
    #cosine_proximity는 안나올 수도 있다.
    model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
    predict_bs_index=model.predict(x_val)
    

    # BS & NLP 넣은 모델 돌리기
    winsize=3
    x_data=[]
    y_data=[]
    msdf_BN = msdf.drop(['usd', 'kbond', 'cd', 'gdp', 'kospi'], axis=1)
    
    for i in range(len(msdf)-winsize):
        x_data.append(msdf_BN.iloc[i:winsize+i,:nlp_len+26])
        y_data.append(msdf_BN.iloc[winsize+i,3])
    x_data=np.array(x_data).astype('float32')
    y_data=np.array(y_data).astype('float32')

    train_num = 550

    x_train = x_data[:train_num]
    x_val = x_data[train_num:]
    y_train = y_data[:train_num]
    y_val = y_data[train_num:]

    model=Sequential()
    model.add(LSTM(128,return_sequences=False, input_shape=(winsize,nlp_len+26,)))
    model.add(Dense(64,activation ="linear"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
    print('predict model:', model.summary())
    #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
    #cosine_proximity는 안나올 수도 있다.
    model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
    predict_bs_nlp=model.predict(x_val)

    
    # INDEX & NLP 넣은 모델 돌리기
    winsize=3
    x_data=[]
    y_data=[]
    msdf_IN = msdf.drop(['f01', 'f03', 'f04', 'f05',
           'f06', 'f07', 'f08', 'f11', 'f12', 'f14', 'f15', 'f21', 'f22', 'f23',
           'f31', 'f33', 'f34', 'f41', 'f42', 'f51',], axis=1)


    for i in range(len(msdf)-winsize):
        x_data.append(msdf_IN.iloc[i:winsize+i,:nlp_len+10])
        y_data.append(msdf_IN.iloc[winsize+i,3])
    x_data=np.array(x_data).astype('float32')
    y_data=np.array(y_data).astype('float32')

    train_num = 550

    x_train = x_data[:train_num]
    x_val = x_data[train_num:]
    y_train = y_data[:train_num]
    y_val = y_data[train_num:]

    model=Sequential()
    model.add(LSTM(128,return_sequences=False, input_shape=(winsize,nlp_len+10,)))
    model.add(Dense(64,activation ="linear"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
    print('predict model:', model.summary())
    #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
    #cosine_proximity는 안나올 수도 있다.
    model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
    predict_index_nlp=model.predict(x_val)

    # BS & INDEX & NLP 넣은 모델 돌리기
    winsize=3
    x_data=[]
    y_data=[]

    for i in range(len(msdf)-winsize):
        x_data.append(msdf.iloc[i:winsize+i,:nlp_len+31])
        y_data.append(msdf.iloc[winsize+i,3])
    x_data=np.array(x_data).astype('float32')
    y_data=np.array(y_data).astype('float32')

    train_num = 550

    x_train = x_data[:train_num]
    x_val = x_data[train_num:]
    y_train = y_data[:train_num]
    y_val = y_data[train_num:]

    model=Sequential()
    model.add(LSTM(128,return_sequences=False, input_shape=(winsize,nlp_len+31,)))
    model.add(Dense(64,activation ="linear"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['cosine_proximity'])
    print('predict model:', model.summary())
    #mse 회귀에서 사용, 다중에선 안된다. 분류모델에만 사용?
    #cosine_proximity는 안나올 수도 있다.
    model.fit(x_train,y_train,epochs=20, validation_data=(x_val, y_val))
    predict_bs_index_nlp=model.predict(x_val)
    
    
    
    
    # 종가
    close_minmax=msdf['sclose'].values[550:]

    # RMSE 산출
    chart_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_chart)).mean())
    bs_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_bs)).mean())
    index_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_index)).mean())
    nlp_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_nlp)).mean())
    bs_index_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_bs_index)).mean())
    bs_nlp_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_bs_nlp)).mean())
    index_nlp_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_index_nlp)).mean())
    bs_index_nlp_RMSE = np.sqrt(np.square(np.subtract(close_minmax,predict_bs_index_nlp)).mean())

    # bs_nlp_RMSE
    # bs_index_nlp_RMSE
    # index_nlp_RMSE
    # nlp_RMSE
    
    # 결과값 데이터프레임화
    df_result.loc[s,:] = [chart_RMSE,bs_RMSE,index_RMSE,nlp_RMSE, bs_index_RMSE, bs_nlp_RMSE, index_nlp_RMSE, bs_index_nlp_RMSE]
    
df_result2 = pd.DataFrame(data=[df_result.mean().values], columns=['chart_RMSE','bs_RMSE','index_RMSE','nlp_RMSE','bs_index_RMSE', 'bs_nlp_RMSE', 'index_nlp_RMSE', 'bs_index_nlp_RMSE'])
df_result2.to_csv('df_result_{}.csv'.format(code), index=False)

C:\Users\pcsea\Anaconda3\envs\study\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\pcsea\Anaconda3\envs\study\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\pcsea\Anaconda3\envs\study\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\pcsea\Anaconda3\envs\study\lib\site-packages\ipykernel_lau

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_26 (LSTM)               (None, 128)               68608     
_________________________________________________________________
dense_50 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 65        
Total params: 76,929
Trainable params: 76,929
Non-trainable params: 0
_________________________________________________________________
predict model: None
Epoch 1/30
18/18 [==============================] - 1s 38ms/step - loss: 0.6230 - cosine_proximity: 1.0000 - val_loss: 0.7335 - val_cosine_proximity: 0.9925
Epoch 2/30
18/18 [==============================] - 0s 11ms/step - loss: 0.6105 - cosine_proximity: 1.0000 - val_loss: 0.7312 - val_cosine_proximity: 0.9925
Epoch 3/30
18/18 [===========================

In [95]:
os.getcwd()

'C:\\Users\\pcsea\\work1\\Self_study\\Company_news_PJ\\fdata\\fighting'

In [94]:
df_result2

,chart_RMSE,bs_RMSE,index_RMSE,nlp_RMSE,bs_index_RMSE,bs_nlp_RMSE,index_nlp_RMSE,bs_index_nlp_RMSE
0,0.176275,0.191187,0.172919,0.169425,0.176685,0.203727,0.165249,0.175075


In [ ]:
# 그래프 그리기
plt.figure(figsize=(60,30))

plt.plot(predict_chart ,label='predict_chart', linewidth='3')
plt.plot(predict_bs ,label='predict_bs',linewidth='3')
plt.plot(predict_bs_index ,label='predict_bs_index',linewidth='3')
plt.plot(predict_index ,label='predict_index',linewidth='3')
plt.plot(close_minmax ,label='close',linewidth='3')


plt.legend(prop={'size':30}, loc='best')
plt.show()

In [ ]:
# 데이터 생성
winsize=3
x_data=[]
y_data=[]
for i in range(len(df)-winsize):
    dump = []
    for j in range(winsize):
        dump += list(df.iloc[i+j,1:32])
    x_data.append(dump)
    y_data.append(df.iloc[winsize+i,30])
# 학습, 테스트 분할
train_num = 550
x_train = x_data[:train_num]
x_test = x_data[train_num:]
y_train = y_data[:train_num]
y_test = y_data[train_num:]
# array로 변환
x_train=np.array(x_train).astype('float32')
y_train=np.array(y_train).astype('float32')
x_test=np.array(x_test).astype('float32')
y_test=np.array(y_test).astype('float32')
# 모델 생성
model = models.Sequential()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(16, activation='sigmoid'))
# 컴파일&예측
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['cosine_proximity'])
history = model.fit(x_train, y_train, epochs=40)
predict_bs=model.predict(x_test)
# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
# 정확도 저장
df_accuracy.loc[l,:] = [code,company,test_acc]

In [ ]:
volumn = stock_volumn_df[:]['volumn'].values
stock = stock_volumn_df[:]['sclose'].values

fig, ax1 = plt.subplots()
ax1.plot(volumn, color='green')

ax2 = ax1.twinx()
ax2.plot(stock, color='deeppink')

plt.show(span_start, span_end)
